# ABS Monthly Household Spending Indicator 5682

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
from abs_data_capture import (
    find_rows,
    AbsLandingPage,
    get_abs_data,
    get_fs_constants,
    metacol,
    get_plot_constants,
)
from plotting import clear_chart_dir, plot_growth_finalise, set_chart_dir

# pandas display settings
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
landing_page = AbsLandingPage(
    theme="economy",
    parent_topic="finance",
    topic="monthly-household-spending-indicator",
)
abs_dict = get_abs_data(landing_page, verbose=True)
source, CHART_DIR, Cat_ID, meta = get_fs_constants(abs_dict, landing_page)
RECENT, plot_times, plot_tags = get_plot_constants(meta)

In get_abs_data() {zip_table=} {verbose=}
About to get data on: Monthly Household Spending Indicator in zip file number: 0
Found: 1 items of type .zip
Found: 18 items of type .xlsx
About to download and cache the latest data.
Saving to cache: ABS_CACHE/monthly-household-spending-indicator--2f3ce5ca1bde4af47c5caec5f09e58fe--All-tables-Experimental-estimates-of-Household-Spending.zip
Extracting DataFrames from the zip-file.
table names: z_name='1' e_name='1' --> r_value='1'
table names: z_name='2' e_name='2' --> r_value='2'
table names: z_name='3' e_name='3' --> r_value='3'
table names: z_name='4' e_name='4' --> r_value='4'
table names: z_name='5' e_name='5' --> r_value='5'
table names: z_name='6' e_name='6' --> r_value='6'
table names: z_name='7' e_name='7' --> r_value='7'
table names: z_name='8' e_name='8' --> r_value='8'
table names: z_name='9' e_name='9' --> r_value='9'
table names: z_name='1' e_name='1' --> r_value='1'
Changing duplicate table name from 1 to 1-9.
table names: z_name

## Plotting

### Set-up

In [3]:
def chart_settings(chart_dir: str = CHART_DIR) -> None:
    """Set up charting parameters."""

    set_chart_dir(chart_dir)
    clear_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")


chart_settings()

### Headline growth

In [4]:
def headline_charts() -> None:
    """Produce headline charts."""

    long_title = 20
    data_sets = {
        "01": "Monthly",
        "01Q": "Quarterly",
    }
    for table, period in data_sets.items():
        if table not in abs_dict:
            continue
        data = abs_dict[table]

        adjustment = "Calendar adjusted"
        prices = "Current Price"
        series_type = "Original"
        base_selector = {
            table: metacol.table,
            series_type: metacol.stype,
            "household spending": metacol.did,
            adjustment: metacol.did,
            prices: metacol.did,
        }
        annual_selector = base_selector | {
            "Through the year percentage change": metacol.did,
        }
        annual_rows = find_rows(meta, annual_selector, verbose=False)
        period_selector = base_selector | {
            f"{period} percentage change": metacol.did,
        }
        period_rows = find_rows(meta, period_selector, verbose=False)

        for chart in [x.split(";")[1].strip() for x in annual_rows[metacol.did]]:
            # data wrangling
            annual_id = annual_rows[
                annual_rows[metacol.did].str.contains(chart, regex=False)
            ][metacol.id].iloc[0]
            period_id = period_rows[
                period_rows[metacol.did].str.contains(chart, regex=False)
            ][metacol.id].iloc[0]

            if annual_id not in data.columns or period_id not in data.columns:
                print(
                    f"Bailing on {chart} where annual={annual_id} and period={period_id}"
                )
                continue

            annual, periodic = data[annual_id], data[period_id]

            # plotting
            separator = " " if len(chart) <= long_title else "\n"
            plot_growth_finalise(
                annual,
                periodic,
                title=f"Growth in Household Spending:{separator}{chart}",
                ylabel="Per cent",
                lfooter=f"Australia. {series_type} series. {prices}. {adjustment}. ",
                rfooter=source,
                show=SHOW,
            )


headline_charts()

## Finished

In [5]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Tue Feb 27 2024 17:50:54

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.1

matplotlib: 3.8.3
pandas    : 2.2.1

Watermark: 2.4.3



In [6]:
print("Done")

Done
